In [1]:
import mne
from ni_eeg_forward_project.util import get_labels_of_points, downsample_leadfield_matrix
import numpy as np

In [2]:
fwd = mne.read_forward_solution("../../neurolib/data/datasets/eeg_fsaverage/fsaverage_fwd_sol/fsaverage_surface_src_fixed_orientation-fwd.fif")

fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True)

Reading forward solution from /Users/martin/Documents/Uni_MSc/Semester04/ni_project/neurolib/ni_eeg_forward_project/playground_dev/../../neurolib/data/datasets/eeg_fsaverage/fsaverage_fwd_sol/fsaverage_surface_src_fixed_orientation-fwd.fif...
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read EEG forward solution (8196 sources, 94 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
    Average patch normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]


In [35]:
if len(fwd_fixed["src"]) == 2:

    # 0 is left, 1 is right hemisphere
    lh = fwd_fixed['src'][0]
    dip_pos_lh = np.vstack(lh['rr'][lh['vertno']])
    rh = fwd_fixed['src'][1]
    dip_pos_rh = np.vstack(rh['rr'][rh['vertno']])

    if dip_pos_lh.shape != dip_pos_rh.shape:
        raise ValueError("Hemispheres ...")

    dip_pos = np.vstack((dip_pos_lh, dip_pos_rh))
else:
    raise ValueError("Other format than expected for a surface source space for whole brain.")

In [37]:
trans_fsaverage = mne.read_trans("/Users/martin/Documents/Uni_MSc/Semester04/ni_project/neurolib/neurolib/data/datasets/"
                           "eeg_fsaverage/fsaverage/bem/fsaverage-trans.fif")
dip_pos_mni = mne.head_to_mni(dip_pos, subject="fsaverage", mri_head_t=trans_fsaverage)

In [43]:
leadfield = fwd_fixed['sol']['data']

assert leadfield.shape[1] == dip_pos_mni.shape[0]

In [45]:
points_found, label_codes, label_strings = get_labels_of_points(dip_pos_mni, atlas="aal2")

In [48]:
ds_everything = downsample_leadfield_matrix(leadfield, label_codes)

IndexError: index 8298 is out of bounds for axis 1 with size 8196

In [52]:
np.unique(label_codes)

array([   0., 2001., 2002., 2101., 2102., 2201., 2202., 2301., 2302.,
       2311., 2312., 2321., 2322., 2331., 2332., 2401., 2402., 2501.,
       2502., 2601., 2602., 2611., 2612., 2701., 2702., 2801., 2802.,
       2811., 2812., 2821., 2822., 2831., 2832., 3001., 3002., 4001.,
       4002., 4011., 4012., 4021., 4022., 4101., 4102., 4111., 4112.,
       4201., 4202., 5001., 5002., 5011., 5012., 5021., 5022., 5101.,
       5102., 5201., 5202., 5301., 5302., 5401., 5402., 6001., 6002.,
       6101., 6102., 6201., 6202., 6211., 6212., 6221., 6222., 6301.,
       6302., 6401., 6402., 7001., 7002., 7012., 7101., 7102., 8101.,
       8102., 8111., 8112., 8121., 8122., 8201., 8202., 8211., 8212.,
       8301., 8302., 9032., 9042.])